In [1]:
import numpy as np
import torch
import tensorflow as tf
import transformers

from decision_transformer import Block

c:\Users\P1361755\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
state_dim = 171
act_dim = 80
hidden_size = state_dim + act_dim
inputs = tf.random.uniform(shape=(1,state_dim))

optimizer = tf.optimizers.Adam()


In [11]:
embed_state = tf.keras.layers.Embedding(state_dim * act_dim + 1,  hidden_size)
test1 = embed_state(inputs)

#embedding got no trainable weights
with tf.GradientTape() as tape:
    gradients = tape.gradient(test1, embed_state.trainable_weights)
    # optimizer.apply_gradients(zip(gradients, embed_state.trainable_weights))
    print(gradients)

test1

[None]


<tf.Tensor: shape=(1, 171, 251), dtype=float32, numpy=
array([[[-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        ...,
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923]]], dtype=float32)>

In [12]:
embed_ln = tf.keras.layers.LayerNormalization(axis = -1)
test2 = embed_ln(test1)

with tf.GradientTape() as tape:
    gradients = tape.gradient(test2, embed_ln.trainable_weights)
    # optimizer.apply_gradients(zip(gradients, embed_state.trainable_weights))
    print(gradients)

test2.shape

[None, None]


TensorShape([1, 171, 251])

In [6]:
transformer = Block(hidden_dim = hidden_size, num_heads = 3, key_dim = state_dim)

# print(transformer.trainable_weights)
with tf.GradientTape() as tape:
    test3 = transformer(test2) #this must be within tape so that it will be under the scope
    gradients = tape.gradient(test3, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, embed_state.trainable_variables))
    print("gradients:", gradients)

# test3

gradients: [<tf.Tensor: shape=(251,), dtype=float32, numpy=
array([ 2.51015854e+01,  9.23164845e+00, -2.47162724e+01,  3.89675369e+01,
       -1.97441273e+01, -3.98406677e+01, -6.39576263e+01, -7.99229097e+00,
        5.48156319e+01,  1.78523941e+01, -3.41536951e+00, -1.18047953e+01,
        1.58236313e+01,  6.93269577e+01, -1.59712934e+01,  7.17024460e+01,
        1.57805414e+01, -4.85543823e+01, -2.40139885e+01, -5.66521797e+01,
       -1.05977936e+01,  1.55910826e+01, -9.92734814e+00,  6.12020922e+00,
        2.58887501e+01, -1.18462059e+02, -2.16151447e+01, -1.37414455e+01,
        4.33698702e+00,  2.07197781e+01,  1.73766060e+01,  3.97143587e-02,
        3.00575466e+01, -1.06804438e+01,  2.73406010e+01,  2.02389870e+01,
       -4.48775558e+01, -4.81777039e+01,  2.43571033e+01,  2.40604820e+01,
       -1.38593245e+01,  1.74764080e+01, -1.99776042e+00,  4.34767609e+01,
       -4.46089096e+01,  1.59361219e+01,  7.19152527e+01,  6.78248596e+01,
       -1.50732975e+01,  3.18005729e+00,

In [1]:
test3.shape

NameError: name 'test3' is not defined

In [15]:
predict_action = tf.keras.Sequential(
   [*([tf.keras.layers.Dense(act_dim)] + 
      [tf.keras.layers.Dense(act_dim, activation = 'tanh')] +
      [tf.keras.layers.Flatten()] +
      [tf.keras.layers.Dense(act_dim)]
      )
   ]
)
predict_action.build((1,171,251))

optimizer = tf.optimizers.Adam()

optimizer.build(predict_action.trainable_variables)

with tf.GradientTape() as tape:
    test4 = predict_action(test3) #this must be within tape so that it will be under the scope
    gradients = tape.gradient(test4, predict_action.trainable_variables)
    optimizer.apply_gradients(zip(gradients, predict_action.trainable_variables))
    print("gradients:", gradients)



gradients: [<tf.Tensor: shape=(251, 80), dtype=float32, numpy=
array([[ 2.3629026 , -1.5478299 , -0.299134  , ...,  0.37742007,
         0.08496375,  1.4312353 ],
       [ 2.3337638 , -1.556642  , -0.293323  , ...,  0.31776902,
         0.09651706,  1.4166787 ],
       [ 2.4275131 , -1.6343493 , -0.29290688, ...,  0.32586277,
         0.10107979,  1.4833175 ],
       ...,
       [-1.2917153 ,  0.8071247 ,  0.18497251, ..., -0.11710357,
         0.16742167, -0.9609259 ],
       [ 2.417749  , -1.694185  , -0.30428913, ...,  0.5763366 ,
        -0.06639215,  1.8385606 ],
       [-2.0241842 ,  0.70327306,  0.55080026, ..., -0.06607601,
         0.482064  , -0.66603327]], dtype=float32)>, <tf.Tensor: shape=(80,), dtype=float32, numpy=
array([ 1.2809769 , -0.86032575, -0.15935689,  1.192809  ,  0.52848315,
       -0.21425591, -0.00674458,  0.27568015,  0.8141367 , -0.24001993,
        0.546695  , -0.06959213, -0.3090031 , -0.8599761 ,  0.06410192,
       -1.013059  , -0.21441837, -0.27671108

In [18]:
# embed_state = tf.keras.layers.Embedding(state_dim * act_dim + 1,  hidden_size)
embed_state = tf.keras.layers.Dense(state_dim)
embed_state_reshape = tf.keras.layers.Reshape((-1,state_dim,1))

embed_ln = tf.keras.layers.LayerNormalization(axis = -1)
transformer = Block(hidden_dim = hidden_size, num_heads = 3, key_dim = state_dim)
predict_action = tf.keras.Sequential(
   [*([tf.keras.layers.Dense(act_dim)] + 
      [tf.keras.layers.Dense(act_dim, activation = 'tanh')] +
      [tf.keras.layers.Reshape((-1, act_dim*state_dim))] +
      [tf.keras.layers.Dense(act_dim)]
      )
   ]
)

model_first = tf.keras.Sequential([
        embed_state,
        embed_state_reshape,
        embed_ln,
       transformer,
       predict_action 
        ])

model_first.build((None,state_dim))

optimizer = tf.optimizers.Adam()
optimizer.build(model_first.trainable_variables)

outputs = tf.random.uniform(shape=(1,act_dim))

with tf.GradientTape() as tape:
    
    test_final = model_first(inputs) #this must be within tape so that it will be under the scope
    loss = tf.math.reduce_mean((test_final - outputs)**2)
    gradients = tape.gradient(loss, model_first.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model_first.trainable_variables))
    print("gradients:", gradients)

test_final.shape


StagingError: Exception encountered when calling layer 'block_12' (type Block).

in user code:

    File "c:\Users\P1361755\OneDrive - Singtel\Desktop\learning_code\quoridor-game-AI\source\dt_models\decision_transformer.py", line 45, in call  *
        attn_output = self.attn(self.ln1(x), training=training)
    File "c:\Users\P1361755\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    StagingError: Exception encountered when calling layer 'causal_self_attention_12' (type CausalSelfAttention).
    
    in user code:
    
        File "c:\Users\P1361755\OneDrive - Singtel\Desktop\learning_code\quoridor-game-AI\source\dt_models\decision_transformer.py", line 18, in call  *
            attn_output = self.mha(
        File "c:\Users\P1361755\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler

In [27]:
model_first.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 251)         3433931   
                                                                 
 layer_normalization_1 (Lay  (None, None, 251)         502       
 erNormalization)                                                
                                                                 
 block (Block)               (None, None, 251)         1023611   
                                                                 
 sequential_8 (Sequential)   (None, 80)                1121120   
                                                                 
Total params: 5579164 (21.28 MB)
Trainable params: 5579164 (21.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
